<a href="https://colab.research.google.com/github/Vspotiv/lab10_jupiter/blob/main/lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ЛР №10. Завдання на програмування №1

The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [ ]:
import pandas as pd
import numpy as np

В цьому модулі уже є реалізована функція read_data, яка читає файл csv та форматує отриману інформацію.

In [ ]:
def read_data():
    df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)
    for col in df.columns:
        if col[:2] == '01':
            df.rename(columns={col: 'Gold'+col[4:]}, inplace=True)
        elif col[:2] == '02':
            df.rename(columns={col: 'Silver'+col[4:]}, inplace=True)
        elif col[:2] == '03':
            df.rename(columns={col: 'Bronze'+col[4:]}, inplace=True)
        elif col[:1] == '№':
            df.rename(columns={col: '#'+col[1:]}, inplace=True)

    names_ids = df.index.str.split('\\s\\(') # split the index by '('

    df.index = names_ids.str[0] # the [0] element is the country name (new index)
    df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

    df = df.drop('Totals')
    #print(df)

    return df

df = read_data()

### first_country(df)

Повинна повертати Series, що містить інформацію щодо першої країни в списку

In [ ]:
def first_country(df):
    return df.iloc[0]

first_country(df) 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Points              2
Name: Afghanistan, dtype: object

### summer_biggest(df)
повинна повертати стрічку з країною, що виграла найбільше золотих медалей в літніх іграх

In [ ]:
def summer_biggest(df):
    return df['Gold'].idxmax()
summer_biggest(df)

'United States'

### difference_biggest(df)

повинна повертати стрічку з країною, що має найбільшу різницю (по модулю)  між кількістю золотих медалей на літніх та зимових іграх

In [ ]:
def difference_biggest(df):

    return (df['Gold'] - df['Gold.1']).abs().idxmax()

difference_biggest(df)

'United States'

### difference_biggest_relative(df)
повинна повертати стрічку з країною, що має найбільшу різницю (по модулю)  між кількістю золотих медалей на літніх та зимових іграх, відносно до загальної кількості виграних золотих медалей цієї країни. Тобто, задіяна така формула:

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Включайте тільки ті країни, які виграли хоча б 1 золоту медаль влітку та взимку.

In [ ]:
def difference_biggest_relative(df):
    df = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    return abs((df['Gold'] - df['Gold.1']) / df['Gold.2']).idxmax()

difference_biggest_relative(df)

'Bulgaria'

### get_points(df)

повинна додати у df стовпець Points, а також повернути тільки його (об’єкт Series). Цей стовпець буде містити кількість очок для кожної країни. За кожну золоту медаль країні присвоюється 3 бали, за кожну срібну - 2 бали і за 

бронзову - 1. Загальна кількість балів - сума балів.


In [ ]:
def get_points(df):
    df['Points'] = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']
    return df['Points']

get_points(df)

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: Points, Length: 146, dtype: int64

# ЛР №10. Завдання на програмування №2

В цьому завданні використовуються дані перепису населення зібрані Бюро перепису населення США. Округи - це політичні та географічні підрозділи штатів США. Цей набір даних містить дані про населення округів і штатів США з 2010 по 2015 рік у файлі census.csv

Потрібно реалізувати модуль counties_process.py, який містить наступні функції:


In [ ]:
path_to_file = '/content/census.csv'

###read_data(path_to_file) 

читає файл та повертає DataFrame об'єкт, без додаткової обробки даних

In [ ]:
"""Analyse census data collected by the US Census Bureau """
import pandas as pd
def read_data(path_to_file):
    """
    Write csv file to DataFrame
    """
    return pd.read_csv(path_to_file)
pd = read_data(path_to_file)

### max_counties(df)
 повертає стрічку - назву штату з найбільшою кількістю округів


In [ ]:
def max_counties(dataf):
    """
    return string with name of state with max COUNTY
    >>> max_counties(read_data('census.csv'))
    'Texas'
    """
    dataf = dataf.mask(dataf['SUMLEV'] != 50)
    return dataf['STNAME'].value_counts().idxmax()
max_counties(pd)

'Texas'

### max_difference(df)
повертає стрічку - назву округу, який містить найбільшу абсолютну зміну числа популяції протягом 2010-2015 років. (Підказка: значення кількості населення зберігаються у стовпцях POPESTIMATE2010 - POPESTIMATE2015, вам потрібно врахувати всі шість стовпців). Наприклад, якщо населення округу за 6-річний період становить 100, 120, 80, 105, 100, 130, то найбільшою його зміною за цей період буде | 130-80 | = 50.

In [ ]:
def max_difference(dataf):
    """
    Return string - name of City with maximum change of population
    during 2015-2010
    >>> max_difference(read_data('census.csv'))
    'Harris County'
    """
    dataf = dataf.mask(dataf['SUMLEV'] != 50)
    max_val = dataf.iloc[:, 9:15].max(axis = 1)
    min_val = dataf.iloc[:, 9:15].min(axis = 1)
    ind = abs(max_val-min_val).idxmax()
    return dataf.at[ind, 'CTYNAME']
max_difference(pd)

'Harris County'

### conditional_counties(df)
У цьому файлі дані розділено на чотири регіони за допомогою колонки "REGION". Створіть функцію, яка знайде округи, що належать до регіонів 1 або 2, назва яких починається на 'Washington', а POPESTIMATE2015 перевищував POPESTIMATE2014. Ця функція повинна повертати 5x2 DataFrame зі стовпцями = ['STNAME', 'CTYNAME']

In [ ]:
def conditional_counties(dataf):
    """
    Make DataFrame with specififc value
    >>> conditional_counties(read_data('census.csv'))
                STNAME            CTYNAME
    896           Iowa  Washington County
    1419     Minnesota  Washington County
    2345  Pennsylvania  Washington County
    2355  Rhode Island  Washington County
    3163     Wisconsin  Washington County
    """
    dataf = dataf.mask(dataf['SUMLEV'] != 50)
    dataf = dataf[(dataf['REGION'] == 1) | (dataf['REGION'] == 2)]
    dataf = dataf.loc[(dataf['CTYNAME'].apply(lambda x: x.startswith("Washington")))\
     & (dataf['POPESTIMATE2015'] > dataf['POPESTIMATE2014'])]
    return dataf.loc[:, ['STNAME', 'CTYNAME']]
conditional_counties(pd)

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
